# Day 15: Chiton

## Part One

<p>You've almost reached the exit of the cave, but the walls are getting closer together. Your submarine can barely still fit, though; the main problem is that the walls of the cave are covered in <a href="https://en.wikipedia.org/wiki/Chiton" target="_blank">chitons</a>, and it would be best not to bump any of them.</p>
<p>The cavern is large, but has a very low ceiling, restricting your motion to two dimensions. The shape of the cavern resembles a square; a quick scan of chiton density produces a map of <em>risk level</em> throughout the cave (your puzzle input). For example:</p>
<pre><code>1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581
</code></pre>
<p>You start in the top left position, your destination is the bottom right position, and you <span title="Can't go diagonal until we can repair the caterpillar unit. Could be the liquid helium or the superconductors.">cannot move diagonally</span>. The number at each position is its <em>risk level</em>; to determine the total risk of an entire path, add up the risk levels of each position you <em>enter</em> (that is, don't count the risk level of your starting position unless you enter it; leaving it adds no risk to your total).</p>
<p>Your goal is to find a path with the <em>lowest total risk</em>. In this example, a path with the lowest total risk is highlighted here:</p>
<pre><code><b>1</b>163751742
<b>1</b>381373672
<b>2136511</b>328
369493<b>15</b>69
7463417<b>1</b>11
1319128<b>13</b>7
13599124<b>2</b>1
31254216<b>3</b>9
12931385<b>21</b>
231194458<b>1</b>
</code></pre>
<p>The total risk of this path is <code><em>40</em></code> (the starting position is never entered, so its risk is not counted).</p>
<p><em>What is the lowest total risk of any path from the top left to the bottom right?</em></p>

---

In [1]:
# Initialise
import numpy as np
from queue import PriorityQueue

with open('Day15.in') as f:
    inputs = np.array([list(i[:-1]) for i in f.readlines()]).astype('int')
    shape = inputs.shape
    start = (0,0)
    finish = (shape[0]-1, shape[1]-1)

In [2]:
# The canonical vectors for adjacency
deltas = [
    ( 1, 0),
    (-1, 0),
    ( 0, 1),
    ( 0,-1)
]

def adjacent(x, y, shape=shape):
    """
    Return all adjacent coordinates given shape
    """
    return [
        (x+i, y+j) for i,j in deltas if (0 <= x+i < shape[0]) and (0 <= y+j < shape[1])
    ]

class Path:
    """
    This class manages all the logic for the paths for the A* algorithm to work.
    """
    def __init__(self, end, visited:set=None, vals:list=None, map_=inputs):
        if visited is None:
            self.visited = {end}
        else:
            self.visited = set(visited)
        
        if vals is None:
            self.vals = [0]
        else:
            self.vals = list(vals)
        
        self.map = map_
        self.end = end
    
    def __len__(self):
        return len(self.vals)
    
    def __contains__(self, item):
        return item in self.visited
    
    def __str__(self):
        return f"Path(len={len(self)}, {self.end})"
    
    def __eq__(self, other):
        return (self.end == other.end) and (self.visited == other.visited)
    
    def __lt__(self, other):
        return self.end < other.end
    
    def __hash__(self):
        return hash(tuple(sorted(self.visited)) + (self.end,))
    
    def __repr__(self):
        return str(self)
    
    def end(self):
        return self.end
    
    def risk(self):
        return sum(self.vals)
    
    def min_total_risk(self):
        return self.risk() + (finish[0] - self.end[0]) + (finish[1] - self.end[1])
    
    def adjacent(self, ends=()):
        return set([
            Path(end=x, visited=self.visited|{x}, vals=self.vals+[self.map[x]], map_=self.map) 
            for x in adjacent(*self.end, shape=self.map.shape) if (x not in self)
        ])
    
    def generate_map(self):
        s = '     ' + ''.join([str(i // 10) for i in range(shape[0])]) + '\n'
        s += '     ' + ''.join([str(i % 10) for i in range(shape[0])]) + '\n'

        for i in range(shape[1]):
            s += str(i).rjust(3) + '  '
            for j in range(shape[0]):
                s += '#' if (i,j) in self.visited else ' '
            s += '\n'
        print(s)

def find_best_path(start, finish, cave):
    """
    Take start, finish and cave map to return the best path
    """
    # Initialise
    p0 = Path(end=start, map_=cave)
    paths = PriorityQueue()
    paths.put((p0.min_total_risk(), p0))
    ends = {(0,0): p0}
        
    while not paths.empty():
        # Get path with minimum possible total risk
        p = paths.get()[1]
        
        # Explore, if end not in ends add new path to ends dict
        # If end in ends then compare and replace if necessary
        for n in p.adjacent():
            if n.end not in ends:
                paths.put((n.min_total_risk(), n))
                ends[n.end] = n
            elif n.risk() < ends[n.end].risk():
                paths.put((n.min_total_risk(), n))
                ends[n.end] = n
    
    # Return best path
    if finish in ends:
        return ends[finish]
    else:
        raise Exception("Something went wrong")

# Solution
print(f"Minimum risk: {find_best_path(start, finish, inputs).risk()}")

Minimum risk: 498


---

## Part Two

<p>Now that you know how to find low-risk paths in the cave, you can try to find your way out.</p>
<p>The entire cave is actually <em>five times larger in both dimensions</em> than you thought; the area you originally scanned is just one tile in a 5x5 tile area that forms the full map. Your original map tile repeats to the right and downward; each time the tile repeats to the right or downward, all of its risk levels <em>are 1 higher</em> than the tile immediately up or left of it. However, risk levels above <code>9</code> wrap back around to <code>1</code>. So, if your original map had some position with a risk level of <code>8</code>, then that same position on each of the 25 total tiles would be as follows:</p>
<pre><code>8 9 1 2 3
9 1 2 3 4
1 2 3 4 5
2 3 4 5 6
3 4 5 6 7
</code></pre>
<p>Each single digit above corresponds to the example position with a value of <code>8</code> on the top-left tile. Because the full map is actually five times larger in both dimensions, that position appears a total of 25 times, once in each duplicated tile, with the values shown above.</p>
<p>Here is the full five-times-as-large version of the first example above, with the original map in the top left corner highlighted:</p>
<pre><code><b>1163751742</b>2274862853338597396444961841755517295286
<b>1381373672</b>2492484783351359589446246169155735727126
<b>2136511328</b>3247622439435873354154698446526571955763
<b>3694931569</b>4715142671582625378269373648937148475914
<b>7463417111</b>8574528222968563933317967414442817852555
<b>1319128137</b>2421239248353234135946434524615754563572
<b>1359912421</b>2461123532357223464346833457545794456865
<b>3125421639</b>4236532741534764385264587549637569865174
<b>1293138521</b>2314249632342535174345364628545647573965
<b>2311944581</b>3422155692453326671356443778246755488935
22748628533385973964449618417555172952866628316397
24924847833513595894462461691557357271266846838237
32476224394358733541546984465265719557637682166874
47151426715826253782693736489371484759148259586125
85745282229685639333179674144428178525553928963666
24212392483532341359464345246157545635726865674683
24611235323572234643468334575457944568656815567976
42365327415347643852645875496375698651748671976285
23142496323425351743453646285456475739656758684176
34221556924533266713564437782467554889357866599146
33859739644496184175551729528666283163977739427418
35135958944624616915573572712668468382377957949348
43587335415469844652657195576376821668748793277985
58262537826937364893714847591482595861259361697236
96856393331796741444281785255539289636664139174777
35323413594643452461575456357268656746837976785794
35722346434683345754579445686568155679767926678187
53476438526458754963756986517486719762859782187396
34253517434536462854564757396567586841767869795287
45332667135644377824675548893578665991468977611257
44961841755517295286662831639777394274188841538529
46246169155735727126684683823779579493488168151459
54698446526571955763768216687487932779859814388196
69373648937148475914825958612593616972361472718347
17967414442817852555392896366641391747775241285888
46434524615754563572686567468379767857948187896815
46833457545794456865681556797679266781878137789298
64587549637569865174867197628597821873961893298417
45364628545647573965675868417678697952878971816398
56443778246755488935786659914689776112579188722368
55172952866628316397773942741888415385299952649631
57357271266846838237795794934881681514599279262561
65719557637682166874879327798598143881961925499217
71484759148259586125936169723614727183472583829458
28178525553928963666413917477752412858886352396999
57545635726865674683797678579481878968159298917926
57944568656815567976792667818781377892989248891319
75698651748671976285978218739618932984172914319528
56475739656758684176786979528789718163989182927419
67554889357866599146897761125791887223681299833479
</code></pre>
<p>Equipped with the full map, you can now find a path from the top left corner to the bottom right corner with the lowest total risk:</p>
<pre><code><b>1</b>1637517422274862853338597396444961841755517295286
<b>1</b>3813736722492484783351359589446246169155735727126
<b>2</b>1365113283247622439435873354154698446526571955763
<b>3</b>6949315694715142671582625378269373648937148475914
<b>7</b>4634171118574528222968563933317967414442817852555
<b>1</b>3191281372421239248353234135946434524615754563572
<b>1</b>3599124212461123532357223464346833457545794456865
<b>3</b>1254216394236532741534764385264587549637569865174
<b>1</b>2931385212314249632342535174345364628545647573965
<b>2</b>3119445813422155692453326671356443778246755488935
<b>2</b>2748628533385973964449618417555172952866628316397
<b>2</b>4924847833513595894462461691557357271266846838237
<b>324</b>76224394358733541546984465265719557637682166874
47<b>15</b>1426715826253782693736489371484759148259586125
857<b>4</b>5282229685639333179674144428178525553928963666
242<b>1</b>2392483532341359464345246157545635726865674683
246<b>1123532</b>3572234643468334575457944568656815567976
423653274<b>1</b>5347643852645875496375698651748671976285
231424963<b>2342</b>5351743453646285456475739656758684176
342215569245<b>332</b>66713564437782467554889357866599146
33859739644496<b>1</b>84175551729528666283163977739427418
35135958944624<b>61</b>6915573572712668468382377957949348
435873354154698<b>44</b>652657195576376821668748793277985
5826253782693736<b>4</b>893714847591482595861259361697236
9685639333179674<b>1</b>444281785255539289636664139174777
3532341359464345<b>2461</b>575456357268656746837976785794
3572234643468334575<b>4</b>579445686568155679767926678187
5347643852645875496<b>3</b>756986517486719762859782187396
3425351743453646285<b>4564</b>757396567586841767869795287
4533266713564437782467<b>554</b>8893578665991468977611257
449618417555172952866628<b>3163</b>9777394274188841538529
462461691557357271266846838<b>2</b>3779579493488168151459
546984465265719557637682166<b>8</b>7487932779859814388196
693736489371484759148259586<b>125</b>93616972361472718347
17967414442817852555392896366<b>6413</b>91747775241285888
46434524615754563572686567468379<b>7</b>67857948187896815
46833457545794456865681556797679<b>26</b>6781878137789298
645875496375698651748671976285978<b>21</b>873961893298417
4536462854564757396567586841767869<b>7</b>952878971816398
5644377824675548893578665991468977<b>6112</b>579188722368
5517295286662831639777394274188841538<b>5</b>299952649631
5735727126684683823779579493488168151<b>4</b>599279262561
6571955763768216687487932779859814388<b>1</b>961925499217
7148475914825958612593616972361472718<b>34725</b>83829458
28178525553928963666413917477752412858886<b>3</b>52396999
57545635726865674683797678579481878968159<b>2</b>98917926
57944568656815567976792667818781377892989<b>24</b>8891319
756986517486719762859782187396189329841729<b>1431</b>9528
564757396567586841767869795287897181639891829<b>2</b>7419
675548893578665991468977611257918872236812998<b>33479</b>
</code></pre>
<p>The total risk of this path is <code><em>315</em></code> (the starting position is still never entered, so its risk is not counted).</p>
<p>Using the full map, <em>what is the lowest total risk of any path from the top left to the bottom right?</em></p>

---

In [3]:
# Define map for entire cave
entire_cave = np.block([
    [((inputs + i + j - 1) % 9) + 1 for i in range(5)] for j in range(5)
])

# Reinitialise shape, finish
shape = entire_cave.shape
finish = (shape[0]-1, shape[1]-1)

# Solution
# This is pretty sub-optimal - need to choose better exploration technique for speed
print(f"Minimum risk: {find_best_path(start, finish, entire_cave).risk()}")

Minimum risk: 2901


---